# Rotate ROS Map

Based on a gold standard map, i.e., a map correctly oriented, rotated a new map by using the features presented in both maps. In order to do so, we use [ORB](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html) to detect features and [Homography](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780), i.e, a 3x3 matrix transformation, to map the points in one image to the corresponding points in the other image. 

In [2]:
import cv2
import numpy as np

In [3]:
# Read images
# Gold standard map
fgoldmap = 'images/rotated_map.png'
goldmap = cv2.imread(fgoldmap)
# Map to align
fimagemap = 'images/map.png'
imagemap = cv2.imread(fimagemap)
print('Map shape: (%d, %d, %d) ' % goldmap.shape)
print('Image shape: (%d, %d, %d) ' % imagemap.shape)

Map shape: (400, 400, 3) 
Image shape: (400, 400, 3) 


<div class="row">
    <div class="column" style="float: left; width: 50%; padding: 5px;">
        Gold standard map:<br>
        <img src="images/rotated_map.png" style="width:70%">
    </div>
    <div class="column" style="float: right; width: 50%; padding: 5px;">
        Map to align:<br>
        <img src="images/map.png" style="width:70%">
    </div>
</div>

In [4]:
MAX_FEATS = 300
MATCH_PERCENT = 0.5

def align_images(image, goldimg):
    # Convert images to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_goldimg = cv2.cvtColor(goldimg, cv2.COLOR_BGR2GRAY)
   
    # Use ORB to detect features
    orb = cv2.ORB_create(MAX_FEATS)
    keypts_image, desc_image = orb.detectAndCompute(gray_image, None)
    keypts_goldimg, desc_goldimg = orb.detectAndCompute(gray_goldimg, None)
   
    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(desc_image, desc_goldimg, None)
   
    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
 
    # Remove not so good matches
    nb_matches = int(len(matches) * MATCH_PERCENT)
    matches = matches[:nb_matches]
 
    # Draw top matches
    img_matches = cv2.drawMatches(image, keypts_image, goldimg, keypts_goldimg, matches, None)
    cv2.imwrite("out/matches.png", img_matches)
   
    # Extract location of good matches
    points_image = np.zeros((len(matches), 2), dtype=np.float32)
    points_goldimg = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points_image[i, :] = keypts_image[match.queryIdx].pt
        points_goldimg[i, :] = keypts_goldimg[match.trainIdx].pt
   
    # Find homography
    h, mask = cv2.findHomography(points_image, points_goldimg, cv2.RANSAC)
 
    # Use homography
    height, width, channels = goldimg.shape
    aligned = cv2.warpPerspective(image, h, (width, height), borderValue=(205, 205, 205))
    return aligned, h

alignedmap, h = align_images(imagemap, goldmap)

In [5]:
outFilename = "out/aligned.png"
print("Saving aligned image : %s" % outFilename) 
cv2.imwrite(outFilename, alignedmap)

# Print estimated homography
print("Estimated homography :")
print(h)

Saving aligned image : out/aligned.png
Estimated homography :
[[ 8.86878458e-01 -4.32318043e-01  1.24927141e+02]
 [ 4.27995137e-01  8.89726206e-01 -5.10845821e+01]
 [-4.32292492e-05  9.03679428e-06  1.00000000e+00]]


<div class="row">
    <div class="column" style="float: left; width: 70%; padding: 5px;">
        Matches between images:<br>
        <img src="images/matches.png" style="width:85%">
    </div>
    <div class="column" style="float: right; width: 30%; padding: 5px;">
        Aligned Map:<br>
        <img src="images/aligned.png" style="width:100%">
    </div>
</div>

# Finding the rotation angle from Homography matrix

In [6]:
import math
theta = - math.atan2(h[0,1], h[0,0]) * 180 / math.pi
print('The rotation angle is: %f' % theta)

The rotation angle is: 25.987396
